In [1]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
import numpy as np

class Gait3DDataset(Dataset):
    """
    Custom PyTorch Dataset for the CASIA-B gait dataset.
    This class loads data in sequences (e.g., 30 frames) for a 3D CNN.
    """
    def __init__(self, root_dir, is_train, sequence_length=30, transform=None):
        self.root_dir = root_dir
        self.is_train = is_train
        self.sequence_length = sequence_length
        self.transform = transform
        
        self.train_sequences = {'nm-01', 'nm-02', 'nm-03', 'nm-04', 'bg-01', 'cl-01'}
        self.test_sequences = {'nm-05', 'nm-06', 'bg-02', 'cl-02'}
        
        self.sequence_paths = []
        self.labels = []
        
        self._load_dataset()

    def _load_dataset(self):
        data_type = 'training' if self.is_train else 'testing'
        print(f"📁 Scanning {data_type} data from: {self.root_dir}")
        target_sequences = self.train_sequences if self.is_train else self.test_sequences

        if not os.path.isdir(self.root_dir):
            raise FileNotFoundError(f"❌ Error: Directory not found at '{self.root_dir}'")

        all_subject_ids = sorted([d for d in os.listdir(self.root_dir) if os.path.isdir(os.path.join(self.root_dir, d))])
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(all_subject_ids)

        for subject_id in all_subject_ids:
            subject_path = os.path.join(self.root_dir, subject_id)
            for sequence_type in sorted(os.listdir(subject_path)):
                if sequence_type in target_sequences:
                    sequence_path = os.path.join(subject_path, sequence_type)
                    for angle in sorted(os.listdir(sequence_path)):
                        angle_path = os.path.join(sequence_path, angle)
                        if not os.path.isdir(angle_path): continue
                        
                        frame_files = [f for f in os.listdir(angle_path) if f.lower().endswith('.png')]
                        
                        if len(frame_files) > 0:
                            self.sequence_paths.append(angle_path)
                            self.labels.append(subject_id)
                        
        print(f"✅ Found {len(self.sequence_paths)} non-empty sequences for the {data_type} set.")

    def __len__(self):
        return len(self.sequence_paths)

    def __getitem__(self, idx):
        angle_path = self.sequence_paths[idx]
        label_str = self.labels[idx]
        
        frame_files = sorted([os.path.join(angle_path, f) for f in os.listdir(angle_path) if f.lower().endswith('.png')])
        
        if not frame_files:
            print(f"⚠️ Warning: Found empty sequence at {angle_path}. Loading first item instead.")
            return self.__getitem__(0) 
        
        indices = np.linspace(0, len(frame_files) - 1, self.sequence_length, dtype=int)
        selected_frames = [frame_files[i] for i in indices]
        
        sequence = []
        for frame_path in selected_frames:
            image = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)
            if self.transform:
                image = self.transform(image)
            sequence.append(image)
            
        sequence_tensor = torch.cat(sequence, dim=0).unsqueeze(0) 
        
        label = self.label_encoder.transform([label_str])[0]
        
        return sequence_tensor, torch.tensor(label, dtype=torch.long)

# --- Main execution block to create DataLoaders ---
if __name__ == '__main__':
    BATCH_SIZE = 64
    NUM_WORKERS = 0 # Keep at 0 for Jupyter
    
    data_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # ---------------------------------------------------
    # THE FIX IS HERE: Corrected variable name
    # ---------------------------------------------------
    train_dataset = Gait3DDataset(
        root_dir='D:/Study/Coding/Project/GAIT/processed_dataset_72x72', 
        is_train=True, 
        sequence_length=30, 
        transform=data_transform
    )
    
    train_loader = DataLoader(
        dataset=train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=NUM_WORKERS,
        pin_memory=True
    )

    # --- Create Testing Dataset and DataLoader ---
    test_dataset = Gait3DDataset(
        root_dir='D:/Study/Coding/Project/GAIT/processed_dataset_72x72', 
        is_train=False, 
        sequence_length=30, 
        transform=data_transform
    )
    
    test_loader = DataLoader(
        dataset=test_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=NUM_WORKERS,
        pin_memory=True
    )
    
    print("\n--- DataLoaders created successfully ---")
    
    try:
        images, labels = next(iter(train_loader))
        print(f"Batch of sequences shape: {images.shape}")
        print(f"Batch of labels shape:   {labels.shape}")
    except Exception as e:
        print(f"Error loading batch: {e}")

📁 Scanning training data from: D:/Study/Coding/Project/GAIT/processed_dataset_72x72
✅ Found 8091 non-empty sequences for the training set.
📁 Scanning testing data from: D:/Study/Coding/Project/GAIT/processed_dataset_72x72
✅ Found 5402 non-empty sequences for the testing set.

--- DataLoaders created successfully ---
Batch of sequences shape: torch.Size([64, 1, 30, 72, 72])
Batch of labels shape:   torch.Size([64])


In [ ]:
import torch
import torch.nn as nn

class Simple3DCNN(nn.Module):
    """
    A 3D CNN with a more robust classifier to prevent bottlenecks.
    """
    def __init__(self, num_classes):
        super(Simple3DCNN, self).__init__()
        
        # Input shape: [B, 1, 30, 72, 72]
        self.conv_stack = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm3d(16), 
            nn.MaxPool3d(kernel_size=(2, 2, 2)), # Output: [B, 16, 15, 36, 36]
            
            nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm3d(32), 
            nn.MaxPool3d(kernel_size=(2, 2, 2)), # Output: [B, 32, 7, 18, 18]
            
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm3d(64), 
            nn.MaxPool3d(kernel_size=(2, 2, 2))  # Output: [B, 64, 3, 9, 9]
        )
        
        # Calculate the flattened size once
        flat_size = 64 * 3 * 9 * 9 # 15552

        # -----------------------------------------------------------------
        # THE FIX: A deeper classifier to handle the 15,552 features
        # -----------------------------------------------------------------
        self.classifier = nn.Sequential(
            nn.Flatten(), 
            
            nn.Linear(in_features=flat_size, out_features=1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(in_features=1024, out_features=512),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(in_features=512, out_features=num_classes)
        )

    def forward(self, x):
        x = self.conv_stack(x)
        x = self.classifier(x)
        return x

# --- Main block to instantiate the model ---
if __name__ == '__main__':
    # Get the number of classes from the train_dataset
    NUM_CLASSES = len(train_dataset.label_encoder.classes_) 
    
    # Create the model
    model = Simple3DCNN(num_classes=NUM_CLASSES)
    
    # Print the model architecture
    print(model)
    
    # Test with a dummy input tensor
    dummy_input = torch.randn(4, 1, 30, 72, 72) 
    output = model(dummy_input)
    
    print(f"\nShape of the model output: {output.shape}")
    

Simple3DCNN(
  (conv_stack): Sequential(
    (0): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU()
    (2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (5): ReLU()
    (6): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (8): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (9): ReLU()
    (10): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(i

In [3]:
import torch.optim as optim
from tqdm import tqdm # For the progress bar
import time

# ==================================================================
#  1. CONFIGURATION AND SETUP
# ==================================================================
NUM_EPOCHS = 10
LEARNING_RATE = 0.001

# --- Setup Device, Loss Function, and Optimizer ---
# (This assumes your 'model', 'train_loader', and 'test_loader' are ready)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ==================================================================
#  2. TRAINING & EVALUATION LOOP
# ==================================================================
print(f"\n🧠 Starting training for {NUM_EPOCHS} epochs...")
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    # --- Training Phase ---
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    # Use tqdm for a progress bar on the training loop
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Training]")
    
    for images, labels in train_pbar:
        # 'images' now has the shape [B, 1, 30, 72, 72]
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images) # The 3D model processes the 3D data
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        
        train_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

    # --- Validation/Testing Phase ---
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    
    test_pbar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Testing]")
    
    with torch.no_grad():
        for images, labels in test_pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    # --- Print a summary for the epoch ---
    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = 100 * train_correct / train_total
    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = 100 * test_correct / test_total
    
    print(f"\nEpoch {epoch+1} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f}%")
    print(f"  Test Loss:  {avg_test_loss:.4f} | Test Acc:  {test_accuracy:.4f}%")
    print("-" * 60)

total_time = time.time() - start_time
print(f"✅ Finished Training in {total_time/60:.2f} minutes.")

Using device: cuda

🧠 Starting training for 10 epochs...


Epoch 1/10 [Testing]: 100%|██████████| 85/85 [04:17<00:00,  3.03s/it]



Epoch 1 Summary:
  Train Loss: 4.9518 | Train Acc: 1.2483%
  Test Loss:  4.5850 | Test Acc:  3.6283%
------------------------------------------------------------


Epoch 2/10 [Testing]: 100%|██████████| 85/85 [03:39<00:00,  2.58s/it]



Epoch 2 Summary:
  Train Loss: 4.5266 | Train Acc: 3.0528%
  Test Loss:  4.3923 | Test Acc:  5.1648%
------------------------------------------------------------


...


Epoch 8 Summary:
  Train Loss: 3.5480 | Train Acc: 12.6931%
  Test Loss:  3.1060 | Test Acc:  23.8800%
------------------------------------------------------------


Epoch 9/10 [Testing]: 100%|██████████| 85/85 [03:18<00:00,  2.33s/it]



Epoch 9 Summary:
  Train Loss: 3.3903 | Train Acc: 15.0908%
  Test Loss:  2.9402 | Test Acc:  29.5446%
------------------------------------------------------------


Epoch 10/10 [Testing]: 100%|██████████| 85/85 [03:11<00:00,  2.26s/it]


Epoch 10 Summary:
  Train Loss: 3.2821 | Train Acc: 17.5504%
  Test Loss:  2.9954 | Test Acc:  29.0263%
------------------------------------------------------------
✅ Finished Training in 90.11 minutes.
